In [1]:
#pip install selenium

In [2]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError as dke
import time

import pandas as pd

# Задание

Написать программу, которая собирает входящие письма из своего или тестового почтового ящика и сложить данные о письмах в базу данных (от кого, дата отправки, тема письма, текст письма полный)

Логин тестового ящика: study.ai_172@mail.ru

Пароль тестового ящика: NextPassword172#

In [3]:
# Mongo DB, создание коллекции

client = MongoClient('127.0.0.1', 27017)
db = client['emails_database']
emails_coll = db.mail_ru_letters
emails_coll

Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'emails_database'), 'mail_ru_letters')

In [4]:
# Функция добавляет Письмо в созданную коллекцию
def insert_mails(collection, add_mail):
    try:
        collection.insert_one(add_mail)
        print(f"Добавление письма {add_mail['subject']}")
    except dke:
        print(f"Письмо по ссылке {add_mail['_id']} уже существует в базе")

In [5]:
driver = webdriver.Chrome()
#driver = webdriver.Chrome(executable_path='bin/chromedriver')

driver.get("https://mail.ru")
actions = ActionChains(driver)

# Login
login = driver.find_element(By.NAME, 'login')
login.send_keys('study.ai_172@mail.ru')

button = driver.find_element(By.XPATH, '//button[@data-testid="enter-password"]')
button.click()

time.sleep(1)

password = driver.find_element(By.NAME, 'password')
password.send_keys('NextPassword172#')

button = driver.find_element(By.XPATH, '//button[@data-testid="login-to-mail"]')
button.click()

time.sleep(5)


# Collect links 

links_mails = []
last_link_mail = None

while True:
    page_with_letters = driver.find_elements(By.XPATH, '//a[contains(@class, "js-letter-list-item")]')
    last_letter_on_page = page_with_letters[-1]
    last_letter_link_on_page = last_letter_on_page.get_attribute('href')
    
    if last_link_mail == last_letter_link_on_page: 
        break 
        
    for letter in page_with_letters:
        links_mails.append(letter.get_attribute('href'))
        
    last_link_mail = last_letter_link_on_page
    actions.move_to_element(last_letter_on_page).perform()
    
    time.sleep(3)
    
print("Список ссылок писем сохранен")



Список ссылок писем сохранен


In [6]:
# Collect letter data

mails_data = []

for link in links_mails:
    mail = {}
    driver.get(link)
    driver.implicitly_wait(10)
    time.sleep(2)
    mail_from = driver.find_element(By.XPATH, '//span[contains(@class, "letter-contact")]')
    mail_date = driver.find_element(By.XPATH, '//div[contains(@class, "letter__date")]')
    mail_subject = driver.find_element(By.XPATH, '//h2[contains(@class, "thread__subject")]')
    mail_text = driver.find_element(By.XPATH, '//div[contains(@class, "letter-body__body-content")]')
    
    #try:
        
    mail['_id'] = link
    mail['from'] = mail_from.get_attribute("title")
    mail['date'] = mail_date.text
    mail['subject'] = mail_subject.text
    mail['text'] = mail_text.text

    mails_data.append(mail)
    
    # Добавления писем в базу
    insert_mails(emails_coll, mail)

Добавление письма ГАРАНТ. Главное для бухгалтера за день: информация Минфина и ФНС России, новости, правовые консультации от 10.11.2021
Добавление письма ГАРАНТ. Правовая картина дня от 10.11.2021
Добавление письма ГАРАНТ. Ежедневный мониторинг федерального законодательства от 10.11.2021
Добавление письма ✏ Design Systems With Figma Tokens, Right-Clicker Mentality, Tor Browser 11, and more…
Добавление письма ГАРАНТ. Ежедневный мониторинг московского законодательства от 10.11.2021
Добавление письма Коронавирус COVID-19: самое важное от 10.11.2021
Добавление письма Garant. Daily monitoring of the Federal legislation at 10.11.2021
Добавление письма Issue #481: Glassmorphism Effect, Developer Tools Secrets
Добавление письма Топ новостей за 24 часа для вас
Добавление письма Как обезопасить себя от мошенников в «Чёрную пятницу»
Добавление письма Shape of Software, Japanese Fireworks, Book Swap, Figma Tokens, Paradox of Choice
Добавление письма 50-летняя Анна Нетребко перенесла срочную операц

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//span[contains(@class, "letter-contact")]"}
  (Session info: chrome=95.0.4638.69)
Stacktrace:
0   chromedriver                        0x000000010f95c519 __gxx_personality_v0 + 577193
1   chromedriver                        0x000000010f8e8663 __gxx_personality_v0 + 102387
2   chromedriver                        0x000000010f4afd28 chromedriver + 171304
3   chromedriver                        0x000000010f4e4c9c chromedriver + 388252
4   chromedriver                        0x000000010f516b04 chromedriver + 592644
5   chromedriver                        0x000000010f501ddd chromedriver + 507357
6   chromedriver                        0x000000010f5148ab chromedriver + 583851
7   chromedriver                        0x000000010f5020a3 chromedriver + 508067
8   chromedriver                        0x000000010f4d85ae chromedriver + 337326
9   chromedriver                        0x000000010f4d

In [28]:
links_mails

['https://e.mail.ru/inbox/0:16365749441864080636:0/?back=1&afterReload=1',
 'https://e.mail.ru/inbox/0:16365722171360693032:0/?back=1&afterReload=1',
 'https://e.mail.ru/inbox/0:16365670231244638115:0/?back=1&afterReload=1',
 'https://e.mail.ru/inbox/0:16365657361767513173:0/?back=1&afterReload=1',
 'https://e.mail.ru/inbox/0:16365634201505627959:0/?back=1&afterReload=1',
 'https://e.mail.ru/inbox/0:16365619191136570781:0/?back=1&afterReload=1',
 'https://e.mail.ru/inbox/0:16365611791385982306:0/?back=1&afterReload=1',
 'https://e.mail.ru/inbox/0:16365599790348551025:0/?back=1&afterReload=1',
 'https://e.mail.ru/inbox/0:16365575351889347922:0/?back=1&afterReload=1',
 'https://e.mail.ru/inbox/0:16365425070646016931:0/?back=1&afterReload=1',
 'https://e.mail.ru/inbox/0:16365356762139391785:0/?back=1&afterReload=1',
 'https://e.mail.ru/inbox/0:16365302000449317994:0/?back=1&afterReload=1',
 'https://e.mail.ru/inbox/0:16364922500173317391:0/?back=1&afterReload=1',
 'https://e.mail.ru/inbox

In [8]:
df = pd.DataFrame(mails_data)
df

,_id,from,date,subject,text
0,https://e.mail.ru/inbox/0:16365749441864080636...,subscribe@garant.ru,"Сегодня, 2:09",ГАРАНТ. Главное для бухгалтера за день: информ...,Письмо отображается некорректно? Посмотрите ис...
1,https://e.mail.ru/inbox/0:16365722171360693032...,subscribe@garant.ru,"Сегодня, 1:23",ГАРАНТ. Правовая картина дня от 10.11.2021,Письмо отображается некорректно? Посмотрите ис...
2,https://e.mail.ru/inbox/0:16365670231244638115...,subscribe@garant.ru,"Вчера, 23:57",ГАРАНТ. Ежедневный мониторинг федерального зак...,Письмо отображается некорректно? Посмотрите ис...
3,https://e.mail.ru/inbox/0:16365657361767513173...,no-reply@webdesignernews.com,"Вчера, 23:35","✏ Design Systems With Figma Tokens, Right-Clic...",Tor Browser 11.0\nBLOG.TORPROJECT.ORG COM...
4,https://e.mail.ru/inbox/0:16365634201505627959...,subscribe@garant.ru,"Вчера, 22:57",ГАРАНТ. Ежедневный мониторинг московского зако...,Письмо отображается некорректно? Посмотрите ис...
5,https://e.mail.ru/inbox/0:16365619191136570781...,subscribe@garant.ru,"Вчера, 22:31",Коронавирус COVID-19: самое важное от 10.11.2021,Письмо отображается некорректно? Посмотрите ис...
6,https://e.mail.ru/inbox/0:16365611791385982306...,subscribe@garant.ru,"Вчера, 22:19",Garant. Daily monitoring of the Federal legisl...,Garant. Daily monitoring of the Federal legisl...
7,https://e.mail.ru/inbox/0:16365599790348551025...,newsletter@css-weekly.com,"Вчера, 21:59","Issue #481: Glassmorphism Effect, Developer To...","Issue #481 • November 10, 2021\nview..."
8,https://e.mail.ru/inbox/0:16365575351889347922...,noreply@pulse.mail.ru,"Вчера, 21:18",Топ новостей за 24 часа для вас,Новости за 24 часа\n Привет! Это ваша...
9,https://e.mail.ru/inbox/0:16365425070646016931...,noreply@e.geekbrains.ru,"Вчера, 17:08",Как обезопасить себя от мошенников в «Чёрную п...,Привет!\nВсе любят скидки! Особенно когда они ...
